In [1]:
!pip install corner

  Created wheel for corner: filename=corner-2.0.1-cp36-none-any.whl size=11642 sha256=6110ae10b42f082074e4cf42149bf20e970bc40360c8ed8fec75d943af03abcc
  Stored in directory: /root/.cache/pip/wheels/70/d8/e5/e0e7974a2a5757483ea5a180c937041cf6872dc9993d78234a
Successfully built corner


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [11]:
cd drive/My\ Drive/Projects/DeepAE

[Errno 2] No such file or directory: 'drive/My Drive/Projects/DeepAE'
/content/drive/My Drive/Projects/DeepAE


In [4]:
ls

4D_vae/                 nn_utils.py                   reproduce_results.ipynb
custom_models.py        plot_activation_functions.py  reproduce_results.py
find_best_model.py      plots/                        results.csv
InfoVAE.py              processed_data/               tune_model.ipynb
models/                 __pycache__/                  tune_model.py
my_matplotlib_rcparams  README.md                     utils.py
my_matplotlib_style.py  relu_vs_tanh.ipynb            vae.ipynb
my_utils.py             relu_vs_tanh.py               vae.py


In [0]:
import os
import utils
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import corner.corner as corner
import matplotlib.pyplot as plt

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [0]:
import fastai
from fastai import train as tr 
from fastai.callbacks import ActivationStats
from fastai import data_block, basic_train, basic_data

In [0]:
import matplotlib as mpl
import my_matplotlib_style as ms
mpl.rc_file('my_matplotlib_rcparams')

In [0]:
from utils import plot_activations

In [0]:
if torch.cuda.is_available():
  fastai.torch_core.defaults.device = 'cuda'

In [0]:
from my_utils import load_data, make_plots, train_evaluate_model

In [0]:
class AE_3D_500cone_bn_custom(nn.Module):
    def __init__(self, hidden_dim_1, hidden_dim_2, hidden_dim_3, n_features=4):
        super(AE_3D_500cone_bn_custom, self).__init__()
        self.en1 = nn.Linear(n_features, hidden_dim_1)
        self.bn1 = nn.BatchNorm1d(hidden_dim_1)
        self.en2 = nn.Linear(hidden_dim_1,  hidden_dim_2)
        self.bn2 = nn.BatchNorm1d(hidden_dim_2)
        self.en3 = nn.Linear(hidden_dim_2, hidden_dim_3)
        self.bn3 = nn.BatchNorm1d(hidden_dim_3)
        self.en4 = nn.Linear(hidden_dim_3, 3)
        self.bn5 = nn.BatchNorm1d(3)
        self.de1 = nn.Linear(3, hidden_dim_3)
        self.bn6 = nn.BatchNorm1d(hidden_dim_3)
        self.de2 = nn.Linear(hidden_dim_3, hidden_dim_2)
        self.bn7 = nn.BatchNorm1d(hidden_dim_2)
        self.de3 = nn.Linear(hidden_dim_2, hidden_dim_1)
        self.bn8 = nn.BatchNorm1d(hidden_dim_1)
        self.de4 = nn.Linear(hidden_dim_1, n_features)
        self.tanh = nn.Tanh()

    def encode(self, x):
        h1 = self.bn1(self.tanh(self.en1(x)))
        h2 = self.bn2(self.tanh(self.en2(h1)))
        h3 = self.bn3(self.tanh(self.en3(h2)))
        z = self.en4(h3)
        return z

    def decode(self, x):
        h5 = self.bn6(self.tanh(self.de1(self.bn5(self.tanh(x)))))
        h6 = self.bn7(self.tanh(self.de2(h5)))
        h7 = self.bn8(self.tanh(self.de3(h6)))
        return self.de4(h7)

    def forward(self, x):
        z = self.encode(x)
        return self.decode(z)

    def describe(self):
        pass

In [0]:
num_epochs = 1000
hidden_dim_1 = 200 #Optimized using Grid Search. See tune_model.py
hidden_dim_2 = 300
hidden_dim_3 = 200
learning_rate = 3e-4

In [0]:
model = AE_3D_500cone_bn_custom(hidden_dim_1, hidden_dim_2, hidden_dim_3)
model_name = 'AE_3D_500cone_bn_custom'
loss = train_evaluate_model(model, model_name, num_epochs, learning_rate, hidden_dim_1, hidden_dim_2, hidden_dim_3)
loss 

epoch,train_loss,valid_loss,time
0,0.048909,0.072411,00:07
1,0.038579,0.034792,00:06
2,0.035496,0.035403,00:06
3,0.033533,0.033856,00:06
